In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import solara

ROOT = Path(solara.__file__).parent / "website" / "pages" / "docs" / "content" / "04-tutorial"
path = ROOT / Path("SF_crime_sample.csv.gz")
url = "https://raw.githubusercontent.com/widgetti/solara/master/solara/website/pages/docs/content/04-tutorial/SF_crime_sample.csv"

if path.exists():
    df_crime = pd.read_csv(path)
else:
    df_crime = pd.read_csv(url)

df_crime

In [11]:
import draw_line

draw_line.Page()

Cannot show ipywidgets in text

In [9]:
import pandas as pd
import solara
import plotly.express as px

def get_data():
    df = pd.DataFrame({'x': range(10), 'y': [i*2 for i in range(10)]})
    return df

@solara.component
def draw():
    fig = px.line(get_data(), x = 'x', y = 'y', title='simple line chart', labels={'x': 'x-axis', 'y': 'y-axis'})
    solara.FigurePlotly(fig)

draw()

Cannot show ipywidgets in text

In [ ]:
from typing import Callable, Dict, List, Optional, Tuple, Union, cast


def foo(x: ...):
    return x

def foo2(x=None):
    return x

# if None is not provided, foo3 needs to be called foo3(None) if no arg is provided. Else call foo3()
def foo3(x: Optional[str]=None):
    if x:
        print(f"foo3 called with {x}")
    else:
        print(f"foo3 called with no param")

# if Optional is not used
def foo4(x: str=None):
    if x:
        print(f"foo4 called with {x}")
    else:
        print(f"foo4 called with no param")

print(foo3())    
print(foo4(None))    
print(foo4())   
print(foo(2))
print(foo2(2))

In [11]:
from typing import Callable, TypeVar
import threading

T = TypeVar('T')  # Declare a generic type variable

def process_event(callback: Callable[[threading.Event], T]) -> T:
    event = threading.Event()
    event.set()  # Simulate triggering the event
    return callback(event)  # Call the function with the event

def event_handler(event: threading.Event) -> str:
    if event.is_set():
        return "Event is set!"
    return "Event is not set."

result = process_event(event_handler)
print(result)  # Output: Event is set!


Event is set!


In [2]:
class MyDecorator:
    def __init__(self, func):
        self.func = func

    def __call__(self, *args, **kwargs):
        print(f"before ....")
        result = self.func(*args, **kwargs)
        print(f"after ....")
        return f"returned from __call__ {result}"

@MyDecorator    
def test(name):
    print(f"test method called with {name}")
    return name

#test("my xx")
d = MyDecorator(test)
d("xx")

before ....
before ....
test method called with xx
after ....
after ....


'returned from __call__ returned from __call__ xx'

In [17]:
class MyDecorator:
    def __init__(self, func):
        self.func = func

    def __call__(self, *args, **kwargs):
        print("Before the function call")
        result = self.func(*args, **kwargs)
        print("After the function call")
        return result

@MyDecorator
def greet(name):
    print(f"Hello, {name}!")

greet("Alice")

Before the function call
Hello, Alice!
After the function call


In [ ]:
class MyDecorator:
    def __init__(self, func):
        self.func = func

    def __call__(self, *args, **kwargs):
        print(f"before ....")
        result = self.func(*args, **kwargs)
        print(f"after ....")
        return f"returned from __call__ {result}"

@MyDecorator    
def test(name):
    print(f"test method called with {name}")
    return name

#test("my xx")
d = MyDecorator(test)
d("xx")

before ....
before ....
test method called with xx
after ....
after ....


'returned from __call__ returned from __call__ xx'

# state pattern

In [ ]:
class Node:
    def __init__(self, tag_name, parent=None):
        self.parent = parent
        self.tag_name = tag_name
        self.children = []
        self.text = ""

    def __str__(self):
        if self.text:
            return self.tag_name + ": " + self.text
        else:
            return self.tag_name

class Parser:
    def __init__(self, parse_string):
        self.parse_string = parse_string
        self.root = None
        self.current_node = None
        self.state = FirstTag()
    
    def process(self, remaining_string):
        remaining = self.state.process(remaining_string, self)
        if remaining:
            self.process(remaining)
   
    def start(self):
        self.process(self.parse_string)

class FirstTag:
    def process(self, remaining_string, parser):
        i_start_tag = remaining_string.find("<")
        i_end_tag = remaining_string.find(">")
        tag_name = remaining_string[i_start_tag + 1 : i_end_tag]
        root = Node(tag_name)
        parser.root = parser.current_node = root
        parser.state = ChildNode()
        return remaining_string[i_end_tag + 1 :]
    
class ChildNode:
    def process(self, remaining_string, parser):
        stripped = remaining_string.strip()

        if stripped.startswith("</"):
            parser.state = CloseTag()
        elif stripped.startswith("<"):
            parser.state = OpenTag()
        else:
            parser.state = TextNode()
        
        return stripped

class OpenTag:
    def process(self, remaining_string, parser):
        i_start_tag = remaining_string.find("<")
        i_end_tag = remaining_string.find(">")
        tag_name = remaining_string[i_start_tag + 1 : i_end_tag]
        node = Node(tag_name, parser.current_node)
        parser.current_node.children.append(node)
        parser.current_node = node
        parser.state = ChildNode()

        return remaining_string[i_end_tag + 1 :]
    
class CloseTag:
    def process(self, remaining_string, parser):
        i_start_tag = remaining_string.find("<")
        i_end_tag = remaining_string.find(">")

        assert remaining_string[i_start_tag + 1] == "/"
        tag_name = remaining_string[i_start_tag + 2 : i_end_tag]
        assert tag_name == parser.current_node.tag_name

        parser.current_node = parser.current_node.parent
        parser.state = ChildNode()

        return remaining_string[i_end_tag + 1 :].strip()
    
class TextNode:
    def process(self, remaining_string, parser):
        i_start_tag = remaining_string.find('<')
        text = remaining_string[:i_start_tag]
        parser.current_node.text = text
        parser.state = ChildNode()

        return remaining_string[i_start_tag:]

def parseXml(content: str):
    p = Parser(contents)
    p.start()
    nodes = [p.root]

    while nodes:
        node = nodes.pop(0)
        print(node)
        nodes = node.children + nodes

''' 
if __name__ == "__main__":
    import sys
    with open(sys.argv[1]) as file:
        contents = file.read()
        parseXml(contents)
''' 

contents = """ 
            <book>
                <author>Dusty Phillips</author>
                <publisher>Packt Publishing</publisher>
                <title>Python 3 Object Oriented Programming</title>
                <content>
                    <chapter>
                        <number>1</number>
                        <title>Object Oriented Design</title>
                    </chapter>
                    <chapter>
                        <number>2</number>
                        <title>Objects In Python</title>
                    </chapter>
                </content>
            </book>
        """

parseXml(contents)

In [ ]:
import sqlite3
# Create a connection to the in-memory database
conn = sqlite3.connect(":memory:")


# Create a cursor object
cursor = conn.cursor()

# Create a table
cursor.execute('''CREATE TABLE IF NOT EXISTS people (id INTEGER PRIMARY KEY, name TEXT)''')

# Insert some data
cursor.execute('''INSERT INTO people (name) VALUES (?)''', ("Alice",))
cursor.execute('''INSERT INTO people (name) VALUES (?)''', ("Bob",))

# Commit the changes
conn.commit()